In [1243]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet18

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [1244]:
#задаем random_state
rstate=44
torch.manual_seed(rstate)

In [1245]:
#задаем префикс пути
PATH_PREF='/content/ufa3/My Drive/ufa3'
#монтируем гугл диск
from google.colab import drive 
drive.mount('/content/ufa3')

Drive already mounted at /content/ufa3; to attempt to forcibly remount, call drive.mount("/content/ufa3", force_remount=True).


## Датасет

Прежде чем разбираться с моделями, нам надо в первую очередь разобраться с тем, как грузить датасет. Давайте напишем класс в торче для этого.

In [1246]:
class ImageDataset(Dataset):
    def __init__(self, data_df, transform=None):

        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        # достаем имя изображения и ее лейбл
        image_name, label = self.data_df.iloc[idx]['ID_img'], self.data_df.iloc[idx]['class']

        # читаем картинку. read the image
        image = cv2.imread(f"{PATH_PREF}/train/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно. transform it, if necessary
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label).long()
    
    def __len__(self):
        return len(self.data_df)

In [1247]:
class TestImageDataset(Dataset):
    def __init__(self, data_df, transform=None):
        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        image_name = self.data_df.iloc[idx]['ID_img']
        
        # читаем картинку
        image = cv2.imread(f"{PATH_PREF}/test/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно
        if self.transform:
            image = self.transform(image)
        
        return image
    
    def __len__(self):
        return len(self.data_df)

In [1248]:
# задаем преобразование изображения.

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),#224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),#224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

In [1249]:
# читаем датасет
data_df = pd.read_csv(f"{PATH_PREF}/train.csv")

In [1250]:
# разделим датасет на трейн и валидацию, чтобы смотреть на качество
train_df, valid_df = train_test_split(data_df, test_size=0.2, random_state=rstate)

In [1251]:
train_dataset = ImageDataset(train_df, train_transform)
valid_dataset = ImageDataset(valid_df, valid_transform)

In [1252]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=32, #32
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=2)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=32, #32
                                           pin_memory=True,
                                           num_workers=2)

## Вспомогательные функции

In [1253]:
def crossvalid(res_model=None,criterion=None,optimizer=None,dataset=None,k_fold=5):
    
    train_score = pd.Series()
    val_score = pd.Series()
    
    total_size = len(dataset)
    fraction = 1/k_fold
    seg = int(total_size * fraction)
    for i in range(k_fold):
        trll = 0
        trlr = i * seg
        vall = trlr
        valr = i * seg + seg
        trrl = valr
        trrr = total_size
        
        train_left_indices = list(range(trll,trlr))
        train_right_indices = list(range(trrl,trrr))
        
        train_indices = train_left_indices + train_right_indices
        val_indices = list(range(vall,valr))
        
        train_set = torch.utils.data.dataset.Subset(dataset,train_indices)
        val_set = torch.utils.data.dataset.Subset(dataset,val_indices)
        
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=50, #50
                                          shuffle=True, num_workers=4)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=50, #50
                                          shuffle=True, num_workers=4)
        train_acc = train(res_model,criterion,optimizer,train_loader,val_loader,1)
        train_score.at[i] = train_acc
    
    return train_score,val_score

In [1254]:
def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)
    
    points = np.array(val_history)
    steps = list(range(0, len(train_history) + 1, int(len(train_history) / len(val_history))))[1:]
    
    plt.scatter(steps, val_history, marker='+', s=180, c='orange', label='val', zorder=2)
    plt.xlabel('train steps')
    
    plt.legend(loc='best')
    plt.grid()

    plt.show()

In [1255]:
from os import listdir

from pandas.core.common import random_state
def train(res_model, criterion, optimizer, train_dataloader, test_dataloader, NUM_EPOCH=15, steps=[0]):
#--------------------------------------------
    i_df = pd.read_csv(f"{PATH_PREF}/tst.csv")
    df_list = listdir(f"{PATH_PREF}/test")
    df_list_prep =[]
    for d in df_list:
      df_list_prep.append(d.split('.', 1)[0])
    test_dataset = TestImageDataset(test_df, valid_transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=1, #64
                                           # shuffle=True,
                                           pin_memory=True,
                                           num_workers=2)
#--------------------------------------------
    train_loss_log = []
    val_loss_log = []
    
    train_acc_log = []
    val_acc_log = []
    
    for epoch in tqdm(range(NUM_EPOCH)):
        model.train()
        torch.save(model, f"{PATH_PREF}/model_last.pt")
        train_loss = 0.
        train_size = 0
        
        train_pred = 0.

        for imgs, labels in train_dataloader:
            optimizer.zero_grad()

            imgs = imgs.cuda()
            labels = labels.cuda()

            y_pred = model(imgs)

            loss = criterion(y_pred, labels)
            loss.backward()
            
            train_loss += loss.item()
            train_size += y_pred.size(0)
            train_loss_log.append(loss.data / y_pred.size(0))
            
            train_pred += (y_pred.argmax(1) == labels).sum()

            optimizer.step()

        train_acc_log.append(train_pred / train_size)

        val_loss = 0.
        val_size = 0
        
        val_pred = 0.
        
        model.eval()
        
        with torch.no_grad():
            for imgs, labels in test_dataloader:
                
                imgs = imgs.cuda()
                labels = labels.cuda()
                
                pred = model(imgs)
                loss = criterion(pred, labels)
                
                val_loss += loss.item()
                val_size += pred.size(0)
                
                val_pred += (pred.argmax(1) == labels).sum()
        

        val_loss_log.append(val_loss / val_size)
        val_acc_log.append(val_pred / val_size)
#-------------------------------------------------------
        predicts = []
        for imgs in test_loader:
            imgs = imgs.cuda()
            pred = model(imgs)

            pred_numpy = pred.cpu().detach().numpy()
            for class_obj in pred_numpy:
              index, max_value = max(enumerate(class_obj), key=lambda i_v: i_v[1])
              predicts.append(index)
        current_df = pd.DataFrame([[name, pred] for name, pred in zip(df_list_prep, predicts)], columns=['ID_img', 'class'])
        diff=i_df.merge(current_df,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
        clear_output()
#-------------------------------------------------------
        print('Train loss:', (train_loss / train_size)*100)
        print('Val loss:', (val_loss / val_size)*100)
        print('Train acc:', (train_pred / train_size)*100)
        print('Val acc:', (val_pred / val_size)*100)
#-------------------------------------------------------
        tval=1-len(diff[diff['_merge'] != "both"])/len(i_df);
        print('Total validation:',tval)
        if (max(steps)<=tval):
          torch.save(model, f"{PATH_PREF}/model_max.pt")
        steps.append(tval)
        print('Max validation:',max(steps))
        
#-------------------------------------------------------
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log,steps,current_df

## Модель

In [1256]:
torch.cuda.empty_cache()
torch.manual_seed(rstate)

In [1257]:
# Подргружаем модель

model = models.resnext50_32x4d(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.LeakyReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 32),
    nn.LogSoftmax(dim=1),
  )
model = model.cuda()

#model = torch.load(f"{PATH_PREF}/model_last.pt")
criterion = torch.nn.CrossEntropyLoss()

In [1258]:
df_list = listdir(f"{PATH_PREF}/test")
test_df = pd.DataFrame({'ID_img':df_list})

In [1259]:
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001) #0.001

In [1260]:
train_loss_log, train_acc_log, val_loss_log, val_acc_log,steps,current_df = train(model, 
                                                                 criterion, 
                                                                optimizer, 
                                                                 train_loader, 
                                                                 valid_loader, 
                                                                 10) #15

Train loss: 1.0011450623586982
Val loss: 1.1265143478682282
Train acc: tensor(88.4793, device='cuda:0')
Val acc: tensor(87.1560, device='cuda:0')
Total validation: 0.8488888888888889


100%|██████████| 10/10 [02:37<00:00, 15.75s/it]

Max validation: 0.8488888888888889


In [1261]:
steps

[0,
 0.6622222222222223,
 0.64,
 0.6,
 0.7377777777777778,
 0.8088888888888889,
 0.8133333333333334,
 0.7866666666666666,
 0.8,
 0.76,
 0.8488888888888889]

## Посмотрим метрикинашей итоговой модели на валидации.

In [1262]:
model.eval()
valid_predicts = []

for imgs, _ in tqdm(valid_loader):
    
    imgs = imgs.cuda()
    pred = model(imgs)

    pred_numpy = pred.cpu().detach().numpy()

    for class_obj in pred_numpy:
      index, max_value = max(enumerate(class_obj), key=lambda i_v: i_v[1])
      valid_predicts.append(index)

100%|██████████| 4/4 [00:02<00:00,  1.88it/s]


In [1263]:
valid_df["pred"] = valid_predicts

# Посчитаем accuracy

In [1264]:
from sklearn.metrics import roc_auc_score, roc_curve, auc , accuracy_score

In [1265]:
val_accuracy = accuracy_score(valid_df['class'].values, valid_df['pred'].values)
print(f"Validation accuracy = {val_accuracy}")

Validation accuracy = 0.8807339449541285


## Предскажем для теста

In [1266]:
from os import listdir
df_list = listdir(f"{PATH_PREF}/test")
test_df = pd.DataFrame({'ID_img':df_list})
test_df.shape

(225, 1)

In [1267]:
test_dataset = TestImageDataset(test_df, valid_transform)

In [1268]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=8,#64
                                           pin_memory=True,
                                           num_workers=2)

#Предсказываем нашей моделью результат

In [1269]:
model.eval()
predicts = []

for imgs in tqdm(test_loader):
    
    imgs = imgs.cuda()
    pred = model(imgs)

    for class_obj in pred_numpy:
      index, max_value = max(enumerate(class_obj), key=lambda i_v: i_v[1])
      predicts.append(index)


100%|██████████| 29/29 [00:03<00:00,  8.14it/s]


In [1270]:
#убираем расширения имен файлов
df_list_prep =[]

for d in df_list:
  df_list_prep.append(d.split('.', 1)[0])

In [1271]:
#submit_df = pd.DataFrame([[name, pred] for name, pred in zip(df_list_prep, predicts)], columns=['ID_img', 'class'])

In [1272]:
current_df.to_csv(f"{PATH_PREF}/submit.csv", index=False)